# Test data generation

We will generate test images with uniformly distributed noise, clustering and background-signal contrast properties.

In [ ]:
import numpy as np
from src.image_generator import generate_multiple_images

In [2]:
np.random.seed(0)

SAVING = True
IMAGE_LIST_PATH = '../data/datasets/source_dataset/test.txt'
IMAGE_SAVE_DIR = '../data/datasets/source_dataset/images/test'
LABEL_SAVE_DIR = '../data/datasets/source_dataset/labels/test'

generate_multiple_images(num_images=1000,
                         start_index=2200,
                         image_size=(2472, 2472),
                         snr_range=(2.5, 12.5),
                         clustering_range=(0, 10),
                         bg_color_range=(30, 220),
                         bg_sn_ratio=(1.2, 1.8),
                         fibers=(45, 15),
                         save=SAVING,
                         save_format='all',
                         image_list_file=IMAGE_LIST_PATH,
                         save_dir=IMAGE_SAVE_DIR,
                         label_dir=LABEL_SAVE_DIR,
                         gui=False)

In [ ]:
from src.data_utils import resize_images_dir

image_path = f"../data/datasets/yolo_dataset/images/test/"
json_path = f"../data/datasets/yolo_dataset/images/test/"

resize_images_dir(image_path, 1024)

# Choosing Best SAM checkpoint

In [ ]:
import os
from ultralytics import SAM
from src.model_eval import evaluate_model

In [ ]:
ckpt_dir = '/workspace/pixi-sam2/sam2_logs/configs/sam2.1_training/fiber_segmentation.yaml/checkpoints'

for ckpt in os.listdir(ckpt_dir):
    if not ckpt.endswith('.pt'):
        continue
    if not ckpt.endswith('_sam2.1_t.pt'):
        os.rename(os.path.join(ckpt_dir, ckpt), os.path.join(ckpt_dir, f'{os.path.splitext(ckpt)[0]}_sam2.1_t.pt'))
        ckpt = f'{os.path.splitext(ckpt)[0]}_sam2.1_t.pt'
    print("CURRENT CHECKPOINT:", ckpt)
    model = SAM(os.path.join(ckpt_dir, ckpt))
    evaluate_model(model,
                test_image_dir='/workspace/datasets/sam_dataset/val/images',
                test_annotation_dir='/workspace/datasets/sam_dataset/val/images',
                batch=1,
                use_sam_annots=True,
                output_file=f'{str(ckpt)}_val.csv',
                verbose=False)

# Model Evaluation

In [1]:
from ultralytics import YOLO
from src.model_eval import evaluate_model

In [4]:
model = YOLO('../models/yolo11large.pt')
evaluate_model(model,
               test_image_dir='../data/datasets/sam_dataset/test/images',
               test_annotation_dir='../data/datasets/sam_dataset/test/images',
               batch=1,
               use_sam_annots=True,
               output_file='../data/results/yolo11large.csv')

# Sorting Data

In [1]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv("../data/results/yolo11large.csv")

# Example binning
df['snr_bin'] = pd.cut(df['snr'], bins=[2.5, 5, 7.5, 10, 12.5])
df['cluster_bin'] = pd.cut(df['clustering'], bins=[0, 2, 4, 6, 8, 10])
df['fibre_bin'] = pd.cut(df['num_fibers'], bins=[0, 30, 60, 90, 120])

In [7]:
snr_summary = df.groupby('snr_bin', observed=False)[['f1_score', 'precision', 'recall']].mean().round(3)

# Group by Clustering bin
cluster_summary = df.groupby('cluster_bin', observed=False)[['f1_score', 'precision', 'recall']].mean().round(3)

# Group by Fibre Count bin
fibre_summary = df.groupby('fibre_bin', observed=False)[['f1_score', 'precision', 'recall']].mean().round(3)

# Display
print("SNR Summary:\n", snr_summary)
print("\nClustering Summary:\n", cluster_summary)
print("\nFibre Count Summary:\n", fibre_summary)

SNR Summary:
               f1_score  precision  recall
snr_bin                                  
(2.5, 5.0]       0.889      0.900   0.879
(5.0, 7.5]       0.885      0.894   0.876
(7.5, 10.0]      0.879      0.889   0.869
(10.0, 12.5]     0.880      0.889   0.872

Clustering Summary:
              f1_score  precision  recall
cluster_bin                             
(0, 2]          0.905      0.916   0.895
(2, 4]          0.904      0.915   0.893
(4, 6]          0.886      0.895   0.877
(6, 8]          0.867      0.877   0.858
(8, 10]         0.845      0.852   0.839

Fibre Count Summary:
            f1_score  precision  recall
fibre_bin                             
(0, 30]       0.900      0.913   0.888
(30, 60]      0.883      0.893   0.875
(60, 90]      0.857      0.863   0.850
(90, 120]     0.793      0.785   0.802
